In [1]:
%reload_ext autoreload
%autoreload 2

import sys

sys.path.append('../Customized_Components') #this path is not right for this PC. Maybne this will work?
from bandaged_dolan import DolanJunctionBandage
from houcklab_qubit import DiffTransmonRounded
import Transmon_specifications as transmon

import warnings

import shapely
from CoupledLineTee import CoupledLineTee
from dolan_junction import DolanJunction
from LaunchpadWirebondCustom import LaunchpadWirebondCustom
from shapely.errors import ShapelyDeprecationWarning
from single_pad_transmon_pocket import TransmonPocket_Single

warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning)
from collections import OrderedDict

import Default_res_params as dp
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import copy
import os



# from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee
# from qiskit_metal.qlibrary.terminations.launchpad_wb import LaunchpadWirebond
# import pyEPR as epr
from qiskit_metal import Dict, Headings, MetalGUI, designs, draw
# from qiskit_metal.qlibrary.terminations.open_to_ground import OpenToGround
# from qiskit_metal.qlibrary.terminations.short_to_ground import ShortToGround
# from qiskit_metal.qlibrary.tlines.anchored_path import RouteAnchors
# from qiskit_metal.qlibrary.tlines.mixed_path import RouteMeander, RouteMixed
# from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight
# from qiskit_metal.toolbox_metal.parsing import *


In [2]:
gui = MetalGUI(dp.design)
design = dp.design
design.overwrite_enabled = True
c1, renderer_q3d, q3d = dp.init_q3d_sim()
eig_all, hfss_renderer, hfss = dp.init_hfss_sim()

## Calculate Qubit Params

In [3]:
import Transmon_property as tp

In [4]:
import scqubits as scq
import numpy as np
import matplotlib.pyplot as plt
import astropy.units as u
import astropy.constants as c

from scipy.optimize import root_scalar

In [5]:
ratio = 65
Phi0 = (c.h / (2 * c.e.si))  # Flux quantum in Weber

def inductance_to_cap(L,ratio = ratio):
    EJ = (((Phi0/2/np.pi)**2/L).to(u.J)/c.h).to(u.GHz)
    EC = EJ/ratio
    return (c.e.si**2 / (2 * EC)/c.h).to(u.fF)

def target_function(L, desired_frequency, ratio):
    L = L * u.nH  # Convert L to nanohenrys
    calculated_frequency = calculate_qubit_freq(L, ratio)
    return calculated_frequency - desired_frequency
def calculate_qubit_freq(L,ratio = ratio):
    
    EJ = (((Phi0/2/np.pi)**2/L).to(u.J)/c.h).to(u.GHz)
    EC = EJ/ratio
    ng = 0.0   # Offset charge
    nlev = 10  # Number of energy levels to consider

    # Create a Transmon object
    transmon = scq.Transmon(EJ=EJ.value, EC=EC.value, ng=ng, ncut=30, truncated_dim=nlev)

    # Calculate the eigenenergies
    eigenenergies = transmon.eigenvals()

    # Calculate the transition frequencies (in GHz)
    transition_frequencies = np.diff(eigenenergies)

    # Calculate the anharmonicity (in GHz)
    anharmonicity = transition_frequencies[1] - transition_frequencies[0]
    return transition_frequencies[0]
    
class TransmonQubit():
    def __init__(self, freq, ratio):
        self.freq = freq
        self.ratio = ratio
        self.C = None,
        self.L = None
        self.EJ = None
        self.EC = None
        self.update_params()
        self.alpha = self.EC
        
    def update_params(self):
        self.C, self.L = self.qubit_freq_to_cap()
        Phi0 = (c.h / (2 * c.e.si))  # Flux quantum in Weber

        # Calculate the Josephson energy EJ and charging energy EC
        EJ = (((Phi0/2/np.pi)**2/self.L).to(u.J)/c.h).to(u.GHz)#, equivalencies=u.spectral())  # Convert to GHz
        EC = (c.e.si**2 / (2 * self.C)/c.h).to(u.GHz)
        self.EJ = EJ
        self.EC = EC
        
    def get_params(self):
        return [self.freq, self.ratio, self.qubit_capacitance, self.qubit_inductance, self.EJ, self.EC]
    def get_qcap(self):
        return self.C
    
    def get_qind(self):
        return self.L
    
    def get_EJ(self):
        return self.EJ
    
    def get_EC(self):
        return self.EC
    
    def qubit_freq_to_cap(self, ):
        desired_frequency = self.freq   # Desired frequency in GHz
        ratio = self.ratio
        # Use scipy.optimize.root_scalar to find the root
        def target_function(L, desired_frequency,):
            self.L = L * u.nH  # Convert L to nanohenrys
            self.calculate_qubit_freq()
            calculated_frequency = self.freq.to(u.GHz).value
            return calculated_frequency - desired_frequency
        result = root_scalar(target_function, args=(desired_frequency.to(u.GHz).value,), bracket=[1, 100], method='brentq')

        # Print the result
        if result.converged:
            L_optimal = result.root
            print(f"Optimal inductance L: {L_optimal} nH")
        else:
            print("Root finding did not converge")
        c_optimal = inductance_to_cap(L_optimal*u.nH, ratio = ratio)
        return c_optimal, L_optimal*u.nH
    def calculate_qubit_freq(self,):
        L = self.L
        ratio = self.ratio
        EJ = (((Phi0/2/np.pi)**2/L).to(u.J)/c.h).to(u.GHz)
        #, equivalencies=u.spectral())  # Convert to GHz
        EC = EJ/ratio
        ng = 0.0   # Offset charge
        nlev = 10  # Number of energy levels to consider

        # Create a Transmon object
        transmon = scq.Transmon(EJ=EJ.value, EC=EC.value, ng=ng, ncut=30, truncated_dim=nlev)

        # Calculate the eigenenergies
        eigenenergies = transmon.eigenvals()

        # Calculate the transition frequencies (in GHz)
        transition_frequencies = np.diff(eigenenergies)

        # Calculate the anharmonicity (in GHz)
        anharmonicity = transition_frequencies[1] - transition_frequencies[0]
        self.freq = transition_frequencies[0]*u.GHz
        self.alpha = anharmonicity*u.GHz
    
    

Design specifics

In [6]:
from Transmon_specifications import find_junction_area, find_junction_capacitance

In [7]:
def pad_len_from_C(c):
    slope = (900-500)/(101.367-61.365)
    intercept = 500 - slope*61.365
    
    return slope*c.to(u.fF).value + intercept 
def unitless_find_j_area(L):
    return find_junction_area(L*u.nH,Jc = 0.42*u.uA/u.um**2).to(u.um**2).value

def unitless_find_pad_len(C):
    slope = (900-500)/(101.367-61.365)
    intercept = 500 - slope*61.365
    
    return slope*C + intercept

In [8]:
def convert_capacitance_to_energy(capacitance):
    C = capacitance*u.fF
    return (c.e.si**2 / (2 * C)/c.h).to(u.GHz)

def get_cap_params(caps, cj):
    cap_matrix = copy.deepcopy(caps)
    T = np.array([[1,0,-1,0],
              [0,1,0,0],
              [1,0,1,0],
              [0,0,0,1]])
    cap_matrix[0,2] -= cj
    cap_matrix[2,0] -= cj
    cap_matrix[2,2] += cj
    cap_matrix[0,0] += cj
    differential_cap_matrix = np.linalg.inv(T).T@(cap_matrix)@np.linalg.inv(T)
    inverse = np.linalg.inv(differential_cap_matrix)
    qubit_capacitance = np.reciprocal(inverse)[0,0]
    coupling_capacitance = np.reciprocal(inverse)[0,1]
    Ec = convert_capacitance_to_energy(qubit_capacitance)
    g = 8*convert_capacitance_to_energy(coupling_capacitance).to(u.MHz)
    return [Ec,g,qubit_capacitance,coupling_capacitance]
    

In [9]:
get_cap_params(np.array([[80,0,0,0],[0,80,0,0],[0,0,80,0],[0,0,0,80]]),0.5)

[<Quantity 0.48425573 GHz>, <Quantity 0. MHz>, 40.0, inf]

## Resonator calculations

In [10]:
import astropy.units as u
import astropy.constants as c
import scipy.optimize as optimize

# define constants for LL designs
sub_t = 350*u.um #substrate thickness (Si in this case)
metal_t = 100*u.nm #Deposited metal thickness (Al)
Sc = 67*u.fF/(u.um)**2 #JJ specific capacitance
epsilon = 11.45
W_jj = 200*u.nm #junction width
Z0 = 50*u.Ohm #characteristic impedance
import qiskit_metal.analyses as analyses

In [11]:
freq = 6*u.GHz.si
line_width = 15*u.um
line_gap = 8.30*u.um

Optimal pin-gap ratio for resonator and feedlines

In [12]:
class ReadoutResonator():
    def __init__(self, freq, short_on_one_end = False,optimal_pin = None, optimal_gap = None ):
        self.freq = freq
        self.short_on_one_end = short_on_one_end
        self.substrate_thickness = 350*u.um
        self.film_thickness = 100*u.nm
        if (optimal_gap is None) and (optimal_pin is None):
            self.pin = 15*u.um
            self.optimize_gap()
        elif optimal_gap is None:
            self.pin = optimal_pin
            self.optimize_gap()
        elif optimal_pin is None:
            self.gap = optimal_gap
            self.optimize_pin()
        
        self.param_from_res_freq(freq, short_on_one_end = short_on_one_end)
        self.C = np.pi/2/self.freq/2/np.pi/self.Z0
        
    def get_params(self):
        return [self.L, self.C, self.target_length]
    
    def get_L(self):
        return self.L
    
    def get_C(self):
        return self.C
    
    def get_target_length(self):
        return self.target_length
    
    def param_from_res_freq(self,freq, short_on_one_end = False):
        self.Lk, self.Lext,self.Cl,self.G,self.Z0,self.etf,self.Cstar = analyses.cpw_calculations.lumped_cpw(freq = self.freq.to(u.Hz).value,
                                        line_width=self.pin.to(u.m).value,
                                        line_gap=self.gap.to(u.m).value,
                                        substrate_thickness=self.substrate_thickness.to(u.m).value,
                                        film_thickness=self.film_thickness.to(u.m).value,)
        self.Lk*=u.H
        self.Lext*=u.H
        self.Cl*=u.F/u.m
        
        self.G*=u.S
        self.Z0*=u.Ohm
        self.etf*=u.H/u.m
        self.Cstar*=u.F
        target_length = analyses.cpw_calculations.guided_wavelength(freq = self.freq.to(u.Hz).value,
                                        line_width=self.pin.to(u.m).value,
                                        line_gap=self.gap.to(u.m).value,
                                        substrate_thickness=self.substrate_thickness.to(u.m).value,
                                        film_thickness=self.film_thickness.to(u.m).value,)[0]*u.m
        if short_on_one_end:
            target_length /= 4
        else:
            target_length /= 2
        self.len = target_length
    def optimize_gap(self):
        def f(line_gap):
            return(analyses.cpw_calculations.lumped_cpw(self.freq.to(u.Hz).value,
                                            self.pin.to(u.m).value,
                                            line_gap,
                                            self.substrate_thickness.to(u.m).value,
                                            self.film_thickness.to(u.m).value)[4]-50)
        optimized_gap = optimize.fsolve(f,self.pin.si.value)[0]*u.m
        self.gap = optimized_gap.to(u.um)
    def optimize_pin(self):
        def f(line_width):
            return(analyses.cpw_calculations.lumped_cpw(self.freq.to(u.Hz).value,
                                            line_width,
                                            self.gap.si.value,
                                            self.substrate_thickness.to(u.m).value,
                                            self.film_thickness.to(u.m).value)[4]-50)
        optimized_pin = optimize.fsolve(f,self.gap.si.value)[0]*u.m
        self.pin = optimized_pin.to(u.um)

### dispersive readout function

In [13]:
class DispersiveReadout():
    def __init__(self, qubit, res, T1, c_qr = None, c_tr = None):
        self.qubit = qubit
        self.res = res
        self.pT1 = T1
        self.optimize_cqr()
        self.optimize_ctr()
        
    def get_params(self):
        return [self.qubit, self.res, self.c_qr, self.c_tr]
    
    def get_cqr(self):
        return self.c_qr
    
    def get_ctr(self):
        return self.c_tr
    
    def get_qubit(self):
        return self.qubit
    
    def get_res(self):
        return self.res
    def calc_g(self):
        c_qr = self.c_qr
        Cq = self.qubit.C
        Cr = self.res.C
        wq = self.qubit.freq*np.pi*2
        wr = self.res.freq*np.pi*2
        EJ = self.qubit.EJ
        EC = self.qubit.EC
        return c_qr/Cq*np.sqrt(c.e.si**2*wr/c.hbar/Cr)*(EJ/8/EC)**0.25
    def update_g(self):
        self.g = self.calc_g()
    def calc_kappa(self):
        res = self.res
        c_tr = self.c_tr
        argv = [res.freq.si.value, 
                c_tr.si.value, 
                res.len.si.value, 
                res.pin.si.value, 
                res.gap.si.value, 
                2]
        
        return (analyses.em.kappa_calculation.kappa_in(res.freq.si.value, 
                c_tr.si.value, 
                res.len.si.value, 
                res.pin.si.value, 
                res.gap.si.value, 
                2)*u.Hz).to(u.MHz)
    def update_kappa(self):
        self.kappa = self.calc_kappa()
    def calc_chi(self):
        qubit = self.qubit
        resonator = self.res
        delta = (qubit.freq-resonator.freq)*2*np.pi
        sum_freq = (qubit.freq+resonator.freq)*2*np.pi
        alpha = qubit.EC
        return 2*self.g**2*(alpha/delta/(delta+alpha)+alpha/sum_freq/(sum_freq+alpha))
    def update_chi(self):
        self.chi = self.calc_chi()
    def calc_purcellT1(self,):
        qubit = self.qubit
        res = self.res
        
        delta = (qubit.freq-res.freq)*2*np.pi
        
        purcellGamma = self.g**2*self.kappa/delta**2
        
        return 1/purcellGamma/2/np.pi
    def calc_purcellT1q(self,update_g = True, update_chi = True):
        '''taking kappa = 2*chi for the optimal SNR, can directly get the T1q'''
        c_qr = self.c_qr
        qubit = self.qubit
        res = self.res
        if update_g:
            self.update_g()
        if update_chi:
            self.update_chi()
        delta = (qubit.freq-res.freq)*2*np.pi
        
        kappa_val = self.chi*2
        
        purcellGamma = self.g**2*kappa_val/delta**2
        return 1/purcellGamma/2/np.pi
    def optimize_cqr(self,):
        '''find the optimal coupling capacitance'''
        target_t1 = self.pT1
        c_qr = 1*u.fF
        self.c_qr = c_qr
        while self.calc_purcellT1q().to(u.ms) > 1.01*target_t1:
            c_qr += .1*u.fF
            self.c_qr = c_qr
        return c_qr
    def optimize_ctr(self, target_kappa=None):
        '''find the optimal coupling capacitance'''
        # if target_kappa is None:
        #     target_kappa = self.chi*2
        # c_tr = 1*u.fF
        # self.c_tr = c_tr
        # self.update_kappa()
        # while self.kappa < target_kappa:
        #     c_tr += .1*u.fF
        #     self.c_tr = c_tr
        #     self.update_kappa()
        # return c_tr
        self.c_tr = 1*u.fF
        return self.c_tr


In [14]:
qubit = TransmonQubit(4*u.GHz, ratio = 65)
res = ReadoutResonator(6*u.GHz, short_on_one_end = False)
dr = DispersiveReadout(qubit,res,5*u.ms)

Optimal inductance L: 13.676453476080171 nH


In [15]:
default_options = Dict(
        cut_l='800um',
        cut_h='1200um',
        gap='20um',
        w = '90um',
        l = '900um',
        r = '60um',
        cpw_l = '50um',
        coupling_gap = '40um',
        JJ_cutout_w = '70um',
        JJ_cutout_h = '70um',
        JJ_cutout_r = '150um',
        JJ_c_contact_l = '40um',
        JJ_c_contact_r = '2.5um',
        JJ_c_contact_w = '10um',
        coupling_d = '100um',
        coupling_pad_w = '90um',
        coupling_r = '40um',
        cpw_pin = '10um',
        chip='main',
        resolution = '10',
        junction = 'False',
        orientation = '0',
        JJ_c_contact_shortl = '10um',
        istunnel = 'False')

## Leonding method

In [16]:
def convert_capacitance_to_energy(capacitance):
    C = capacitance*u.fF
    return (c.e.si**2 / (2 * C)/c.h).to(u.GHz)

def get_cap_params_dim4(caps, cj):
    cap_matrix = copy.deepcopy(caps)
    T = np.array([[1,0,0,0],
                  [0,1,1,0],
                  [0,1,-1,0],
                  [0,0,0,1]])
    cap_matrix[1,2] -= cj
    cap_matrix[2,1] -= cj
    cap_matrix[2,2] += cj
    cap_matrix[1,1] += cj
    differential_cap_matrix = np.linalg.inv(T).T@(cap_matrix)@np.linalg.inv(T)
    inverse = np.linalg.inv(differential_cap_matrix)
    qubit_capacitance = np.reciprocal(inverse)[2,2]
    coupling_capacitance = np.reciprocal(inverse)[2,3]
    Ec = convert_capacitance_to_energy(qubit_capacitance)
    J = 8*convert_capacitance_to_energy(coupling_capacitance).to(u.MHz)
    # gp = (1/c.hbar*c.e.si*np.sqrt(np.pi*c.hbar/2/z0)*(32*Ec/Ej)**0.25).to(u.MHz)*coupling_capacitance
    # print(gp)
    return [Ec,J,qubit_capacitance,coupling_capacitance]
def get_cap_params_dim2(caps, cj):
    cap_matrix = copy.deepcopy(caps)
    T = np.array([[1,1,0],
                  [1,-1,0],
                  [0,0,1]])
    cap_matrix[0,1] -= cj
    cap_matrix[1,0] -= cj
    cap_matrix[1,1] += cj
    cap_matrix[0,0] += cj
    differential_cap_matrix = np.linalg.inv(T).T@(cap_matrix)@np.linalg.inv(T)
    inverse = np.linalg.inv(differential_cap_matrix)
    qubit_capacitance = np.reciprocal(inverse)[1,1]
    coupling_capacitance = np.reciprocal(inverse)[1,2]
    Ec = convert_capacitance_to_energy(qubit_capacitance)
    J = 8*convert_capacitance_to_energy(coupling_capacitance).to(u.MHz)
    
    # gp = (1/c.hbar*c.e.si*np.sqrt(np.pi*c.hbar/2/z0)*(32*Ec/Ej)**0.25).to(u.MHz)*coupling_capacitance
    # print(gp)
    return [Ec,J,qubit_capacitance,coupling_capacitance]
def qubit_zpf(E_J, E_c):
    z_Q = (1/2*np.pi)*np.sqrt(8*E_c/E_J)
    return np.sqrt(1/(4*np.pi*z_Q))
def res_zpf(Z_0 = 50*u.Ohm):
    R_Q = c.h/(2*c.e.si**2)
    Z = 2*Z_0/np.pi
    z= Z/R_Q
    return np.sqrt(1/(4*np.pi*z))
    

In [17]:
phi0 = c.h/2/c.e.si
epsilon_r = 11.45
z0 = 50*u.Ohm

def g_leonDing_old(dim = 4, c1 = c1, cj = 2, Lj = 13*u.nH):
    df = c1.sim.capacitance_matrix
    df1 = copy.deepcopy(df)
    df1['ground_main_plane']['ground_main_plane'] += res.C.to(u.fF).value
    df1['resonator_pad_Q1']['resonator_pad_Q1'] += res.C.to(u.fF).value
    df1['resonator_pad_Q1']['ground_main_plane'] -= res.C.to(u.fF).value
    df1['ground_main_plane']['resonator_pad_Q1'] -= res.C.to(u.fF).value
    cap = df1.to_numpy()
    if dim == 3:
        capp = cap[1:,1:]
        Ec, J,_,_ = get_cap_params(capp,cj)
    elif dim == 4:
        capp = cap
        Ec, J,_,_ = get_cap_params_dim4(capp,cj) 
        
    
    # Ec = (c.e.si**2/2/Cq)/c.h
    Ej = ((phi0/2/np.pi)**2/Lj)/c.h   
    g = (qubit_zpf(Ej,Ec)*J*res_zpf()).to(u.MHz)
    
    
    
    return g, Ec, Ej, 

In [18]:
def g_leonDing(dim = 4, c1 = c1, cj = 2, Lj = 13*u.nH):
    df = c1.sim.capacitance_matrix
    df1 = copy.deepcopy(df)
    
    
    # df1['ground_main_plane']['ground_main_plane'] += res.C.to(u.fF).value
    # df1['resonator_pad_Q1']['resonator_pad_Q1'] += res.C.to(u.fF).value
    # df1['resonator_pad_Q1']['ground_main_plane'] -= res.C.to(u.fF).value
    # df1['ground_main_plane']['resonator_pad_Q1'] -= res.C.to(u.fF).value
    cap = df1.to_numpy()
    cap[1,1] += cap[0,1]
    cap[2,2] += cap[0,2]
    cap[3,3] += cap[0,3]
    
    cap[0,3] += res.C.to(u.fF).value
    cap[3,0] += res.C.to(u.fF).value
    cap[0,0] -= res.C.to(u.fF).value
    cap[3,3] -= res.C.to(u.fF).value
    
    
    if dim == 3:
        capp = cap[1:,1:]
        Ec, J,_,_ = get_cap_params(capp,cj)
    elif dim == 4:
        capp = cap
        Ec, J,_,_ = get_cap_params_dim4(capp,cj) 
        
    
    # Ec = (c.e.si**2/2/Cq)/c.h
    Ej = ((phi0/2/np.pi)**2/Lj)/c.h   
    g = (qubit_zpf(Ej,Ec)*J*res_zpf()).to(u.MHz)
    
    
    
    return g, Ec, Ej, 

In [19]:
# g_leonDing(dim = 4, c1 = c1, cj = 2, Lj = 13*u.nH)

In [20]:
qubit.alpha = 0.2*u.GHz
# dr.calc_g().to(u.MHz)

In [21]:
c1.setup.junctions = Dict({'Lj': qubit.L.to(u.nH).value, 'Cj': 2})
c1.setup.freq_readout = 6.0
c1.setup.freq_bus = []

c1.run_lom()
c1.lumped_oscillator_all

02:54PM 52s WARNING [run_lom]: Please initialize the capacitance_matrix before executing this method.`self.sim.capacitance_matrix = pd.DataFrame(...)`


{}

## Alex Page Thesis method

In [22]:
df = c1.sim.capacitance_matrix

In [23]:
phi0 = c.h/2/c.e.si
T = 30*u.mK
epsilon_r = 11.45
z0 = 50*u.Ohm


Cq = 20*u.fF
Lj = 13*u.nH
Ec = (c.e.si**2/2/Cq)/c.h


In [24]:
def g_alexPage(c1 = c1,wr = 6*u.GHz, Lj = 13*u.nH):
    df = c1.sim.capacitance_matrix
    c_1 = -df['pad_left_Q1']['resonator_pad_0_Q1']*u.fF
    c_2 = -df['pad_right_Q1']['ground_main_plane']*u.fF
    c_3 = -df['pad_right_Q1']['resonator_pad_0_Q1']*u.fF
    c_4 = -df['pad_left_Q1']['ground_main_plane']*u.fF
    ct = -df['pad_left_Q1']['pad_right_Q1']*u.fF

    c_phi = (c_2+c_3)*(c_1+c_4)/(c_1+c_2+c_3+c_4)+2*u.fF+ct
    c_ceff = (c_1**2-c_3*c_4)/(c_1+c_2+c_3+c_4+ct)
    beta = (c_1*c_2-c_3*c_4)/((c_2+c_3)*(c_1+c_4)+ct*(c_1+c_2+c_3+c_4))
    Ec = (c.e.si**2/2/c_phi)/c.h
    Ej = ((phi0/2/np.pi)**2/Lj)/c.h
    w01 = np.sqrt(8*Ec*Ej)
    delta = wr-w01
    c_res = np.pi/2/wr/z0
    Vrms = np.sqrt(c.hbar*wr/2/c_res)
    g = c.e.si*Vrms*beta/c.hbar*np.sqrt(2)*(Ej/8/Ec)**0.25
    return g.to(u.MHz), Ec.to(u.MHz), Ej.to(u.GHz), w01.to(u.GHz)

In [25]:
# g_alexPage()

## Qiskit Metal

In [26]:
qubit_layer = 5
junction_layer = 20
ab_layer = 31
ab_square_layer = 30
junction_area_layer = 60

In [27]:
from datetime import datetime

def get_current_time_string():
    now = datetime.now()
    return now.strftime("%Y-%m-%d%H-%M")

# Example usage
print(get_current_time_string())

2025-01-2414-54


In [28]:
qubit = TransmonQubit(4*u.GHz, ratio = 65)
res = ReadoutResonator(6*u.GHz, short_on_one_end = False)

Optimal inductance L: 13.676453476080171 nH


In [29]:
raise Exception('stop here')

Exception: stop here

In [31]:
default_options = Dict(
        cut_l='800um',
        cut_h='1200um',
        gap='20um',
        w = '90um',
        l = '900um',
        r = '60um',
        cpw_l = '50um',
        coupling_gap = '40um',
        JJ_cutout_w = '70um',
        JJ_cutout_h = '70um',
        JJ_cutout_r = '150um',
        JJ_c_contact_l = '40um',
        JJ_c_contact_r = '2.5um',
        JJ_c_contact_w = '10um',
        coupling_d = '100um',
        coupling_pad_w = '90um',
        coupling_r = '40um',
        cpw_pin = '10um',
        chip='main',
        resolution = '10',
        junction = 'False',
        orientation = '0',
        JJ_c_contact_shortl = '10um',
        istunnel = 'False')

### Define qubit, resonator params

In [ ]:
qubit = TransmonQubit(4*u.GHz, ratio = 65)
res = ReadoutResonator(6.5*u.GHz, short_on_one_end = False)



Optimal inductance L: 13.676453476080171 nH


In [ ]:
dr = DispersiveReadout(qubit,res,5*u.ms)

In [ ]:
q1 = DiffTransmonRounded(design,'Q1',options = default_options)
gui.rebuild()

09:56PM 59s ERROR [__getitem__]: 
 Component name = `Q1`
 Option name    = `coupling_arm`
09:56PM 59s ERROR [__getitem__]: 
 Component name = `Q1`
 Option name    = `jj_taper_r`
09:56PM 59s ERROR [rebuild]: ERROR in building component name=Q1, error=unsupported operand type(s) for /: 'NoneType' and 'float'


TypeError: unsupported operand type(s) for /: 'NoneType' and 'float'

In [ ]:
raise Exception('stop!')

Exception: stop!

### Simulation

In [ ]:
raise Exception('stop!')

Exception: stop!

In [ ]:
qubit = TransmonQubit(4*u.GHz, ratio = 65)
res = ReadoutResonator(6*u.GHz, short_on_one_end = False)
# dr = DispersiveReadout(qubit,res,5*u.ms)

Optimal inductance L: 13.676453476080171 nH


In [114]:
# default_options['coupling_d'] = '300um'
# default_options['coupling_gap'] = '15um'
# default_options['gap'] = '25um'
# default_options['l'] = '00um'
# default_options['cut_l'] = '900um'
default_options = default_options_q1
c1, renderer_q3d, q3d = dp.init_q3d_sim()
# sweep_param = 'coupling_d'
sweep_param = 'l'
sweep_array = np.arange(800, 1100,10)
date = get_current_time_string()
dp.change_inductance([f'{(qubit.L.to(u.nH).value):.2f}nH'],eig_all=eig_all,c1 = c1)

for i,l in enumerate(sweep_array):
    default_options[sweep_param] = f'{l}um'
    # default_options['cut_l'] = default_options['coupling_d']+'+500um+'+default_options['coupling_gap']+'+'+default_options['gap']
    q1 = DiffTransmonRounded(design,'Q1',options = default_options)
    gui.rebuild()
    
    c1.sim.run(components = ['Q1'],open_terminations = [('Q1','cpw_stub')])
    print(l)
    print(c1.sim.capacitance_matrix)
    if i ==0:
        dir = 'data/'+date+f'{sweep_param}_capmat'
        os.mkdir(dir)
    cap_mat = c1.sim.capacitance_matrix
    cap_mat.to_csv(dir+f'/{l}.csv')
    
    
    
    # c1.setup.junctions = Dict({'Lj': qubit.L.to(u.nH).value, 'Cj': 2})
    c1.setup.freq_readout = res.freq.to(u.GHz).value
    c1.setup.freq_bus = []

    c1.run_lom()
    result = pd.DataFrame(c1.lumped_oscillator)
    last_row = result
    for keys in default_options.keys():
        last_row[keys] = design.parse_value(default_options[keys])
    last_row['Lj [nH]'] = qubit.L.to(u.nH).value
    
    g_alex,Ec_alex, Ej_alex, wr_alex = g_alexPage(c1 = c1, Lj = qubit.L)
    # g_leon, Ec_leon, Ej_leon = g_leonDing(c1 = c1, Lj = qubit.L)
    # g_leon3, Ec_leon3, Ej_leon3 = g_leonDing(c1 = c1,dim = 3, Lj = qubit.L)
    
    last_row['g_alex [MHz]'] = g_alex.to(u.MHz).value
    last_row['Ec_alex [MHz]'] = Ec_alex.to(u.MHz).value
    last_row['Ej_alex [GHz]'] = Ej_alex.to(u.GHz).value
    last_row['wr_alex [GHz]'] = wr_alex.to(u.GHz).value
    # last_row['g_leon [MHz]'] = g_leon.to(u.MHz).value
    # last_row['Ec_leon [MHz]'] = Ec_leon.to(u.MHz).value
    # last_row['Ej_leon [GHz]'] = Ej_leon.to(u.GHz).value
    
    # last_row['g_leon3 [MHz]'] = g_leon3.to(u.MHz).value
    # last_row['Ec_leon3 [MHz]'] = Ec_leon3.to(u.MHz).value
    # last_row['Ej_leon3 [GHz]'] = Ej_leon3.to(u.GHz).value
    
    dr.g = last_row['gbus'][0]*u.MHz
    
    dr.update_chi()
    chi = dr.chi
    last_row['Chi [MHz]'] = chi.to(u.MHz).value
    qiskit_chi = np.abs(last_row['chi_in_MHz'][0])*u.MHz
    purcell_t1 = dr.calc_purcellT1q(update_g=False)
    target_kappa = 2*chi
    
    dr.chi = qiskit_chi
    purcell_t1_qiskit = dr.calc_purcellT1q(update_g=False, update_chi = False)
    target_kappa_qiskit = 2*chi
    
    
    
    last_row['Purcell T1 [us]'] = purcell_t1.to(u.us).value
    last_row['target kappa [MHz]'] = target_kappa.to(u.MHz).value
    
    last_row['Purcell T1 qiskit [us]'] = purcell_t1_qiskit.to(u.us).value
    last_row['target kappa qiskit [MHz]'] = target_kappa_qiskit.to(u.MHz).value
    
    
    
    
    
    if i ==0:
        df = last_row
    else:
        df = df.append(last_row, ignore_index=True,)

    df.to_csv('data/'+date+f'{sweep_param}_sweep.csv')
    
    if i == len(sweep_array)-1:
        plt.cla()
        plt.scatter(df[sweep_param].values, df['g_alex [MHz]'].values, label = 'g_alex', color = 'red', marker = 'x')
        # plt.scatter(df[sweep_param].values, df['g_leon [MHz]'].values/2, label = 'g_leon', color = 'orange', marker = '.')
        # plt.scatter(df[sweep_param].values, -df['g_leon3 [MHz]'].values, label = 'g_leon3', color = 'green', marker = 'o')
        plt.scatter(df[sweep_param].values, df['gbus'].values, label = 'g_qiskit', color = 'blue', marker = '*')
        plt.xlabel(sweep_param)
        plt.ylabel('g [MHz]')
        plt.legend()
        plt.savefig('data/'+date+sweep_param+f'g_sweep1.png')
        plt.show()
        
        plt.cla()
        plt.scatter(df[sweep_param].values, df['Ec_alex [MHz]'].values, label = 'Ec_alex', color = 'red', marker = 'x')
        # plt.scatter(df[sweep_param].values, df['Ec_leon [MHz]'].values, label = 'Ec_leon', color = 'orange', marker = '.')
        plt.scatter(df[sweep_param].values, df['EC'].values, label = 'Ec_qiskit', color = 'blue', marker = 'o')
        # plt.scatter(df[sweep_param].values, df['Ec_leon3 [MHz]'].values, label = 'Ec_leon3', color = 'green', marker = '*')
        plt.xlabel(sweep_param)
        plt.ylabel('Ec [MHz]')
        plt.legend()
        plt.savefig('data/'+date+sweep_param+f'Ec_sweep.png')
        plt.show()
        
        plt.cla()
        plt.scatter(df[sweep_param].values, df['Ej_alex [GHz]'].values, label = 'Ec_alex', color = 'red', marker = 'x')
        # plt.scatter(df[sweep_param].values, df['Ej_leon [GHz]'].values, label = 'Ec_leon', color = 'orange', marker = '.')
        plt.scatter(df[sweep_param].values, df['EJ'].values, label = 'Ec_qiskit', color = 'blue', marker = 'o')
        plt.xlabel(sweep_param)
        plt.ylabel('Ej [MHz]')
        plt.legend()
        plt.savefig('data/'+date+sweep_param+f'Ej_sweep.png')
        plt.show()
        
        plt.cla()
        plt.scatter(df[sweep_param].values, df['Chi [MHz]'].values, label = 'Chi', color = 'red', marker = 'x')
        plt.scatter(df[sweep_param].values, -df['chi_in_MHz'].values, label = 'Chi_qiskit', color = 'blue', marker = 'o')
        plt.xlabel(sweep_param)
        plt.ylabel('Chi [MHz]')
        plt.legend()
        plt.savefig('data/'+date+sweep_param+f'Chi_sweep.png')
        plt.show()
        
        plt.cla()
        plt.scatter(df[sweep_param].values, df['Purcell T1 [us]'].values, label = 'Purcell T1', color = 'red', marker = 'x')
        plt.scatter(df[sweep_param].values, df['Purcell T1 qiskit [us]'].values, label = 'Purcell T1 qiskit', color = 'blue', marker = 'o')
        plt.xlabel(sweep_param)
        plt.ylabel('Purcell T1 [us]')
        plt.legend()
        plt.savefig('data/'+date+sweep_param+f'Purcell_T1_sweep.png')
        plt.show()
        
        plt.cla()
        plt.scatter(df[sweep_param].values, df['fQ'].values, label = 'qiskit freQ', color = 'red', marker = 'x')
        plt.scatter(df[sweep_param].values, df['wr_alex [GHz]'].values, label = 'alex freq', color = 'blue', marker = 'o')
        plt.xlabel(sweep_param)
        plt.ylabel('fQ [GHz]')
        plt.legend()
        plt.savefig('data/'+date+sweep_param+f'fQ_sweep.png')
        plt.show()
# q1 = DiffTransmonRounded(design,'Q1',options = default_options)
# gui.rebuild()

INFO 02:32PM [connect_design]: 	Opened active design
	Design:    Design_q3d [Solution type: Q3D]
INFO 02:32PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.AnsysQ3DSetup'>)
INFO 02:32PM [analyze]: Analyzing setup Setup
INFO 02:33PM [get_matrix]: Exporting matrix data to (C:\Users\slab\AppData\Local\Temp\tmp24wugwgi.txt, C, , Setup:LastAdaptive, "Original", "ohm", "nH", "fF", "mSie", 5000000000, Maxwell, 1, False
INFO 02:33PM [get_matrix]: Exporting matrix data to (C:\Users\slab\AppData\Local\Temp\tmpxjpxgcmx.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5000000000, Maxwell, 1, False
INFO 02:33PM [get_matrix]: Exporting matrix data to (C:\Users\slab\AppData\Local\Temp\tmptayj3p7_.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5000000000, Maxwell, 2, False
INFO 02:33PM [get_matrix]: Exporting matrix data to (C:\Users\slab\AppData\Local\Temp\tmpvl2esit_.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5000000000, 

800
                    ground_main_plane  pad_left_Q1  pad_right_Q1  \
ground_main_plane           339.30107   -111.86535    -111.39278   
pad_left_Q1                -111.86535    192.82602     -61.40374   
pad_right_Q1               -111.39278    -61.40374     178.08177   
resonator_pad_0_Q1          -16.40672    -15.21850      -0.52739   

                    resonator_pad_0_Q1  
ground_main_plane            -16.40672  
pad_left_Q1                  -15.21850  
pad_right_Q1                  -0.52739  
resonator_pad_0_Q1            32.42705  


INFO 02:33PM [connect_design]: 	Opened active design
	Design:    Design_q3d [Solution type: Q3D]


[1, 2] [3]
Predicted Values

Transmon Properties
f_Q 3.979801 [GHz]
EC 157.591422 [MHz]
EJ 13.616300 [GHz]
alpha -174.043673 [MHz]
dispersion 0.023760 [KHz]
Lq 11.995161 [nH]
Cq 122.914229 [fF]
T1 583.984014 [us]

**Coupling Properties**

tCqbus1 6.846054 [fF]
gbus1_in_MHz 53.983196 [MHz]
χ_bus1 -0.239203 [MHz]
1/T1bus1 272.533047 [Hz]
T1bus1 583.984014 [us]
Bus-Bus Couplings


INFO 02:33PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.AnsysQ3DSetup'>)
INFO 02:33PM [analyze]: Analyzing setup Setup
INFO 02:33PM [get_matrix]: Exporting matrix data to (C:\Users\slab\AppData\Local\Temp\tmp8pra4iqq.txt, C, , Setup:LastAdaptive, "Original", "ohm", "nH", "fF", "mSie", 5000000000, Maxwell, 1, False
INFO 02:33PM [get_matrix]: Exporting matrix data to (C:\Users\slab\AppData\Local\Temp\tmpnai3zyun.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5000000000, Maxwell, 1, False
INFO 02:33PM [get_matrix]: Exporting matrix data to (C:\Users\slab\AppData\Local\Temp\tmplem32llw.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5000000000, Maxwell, 2, False
INFO 02:33PM [get_matrix]: Exporting matrix data to (C:\Users\slab\AppData\Local\Temp\tmph97n5qun.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5000000000, Maxwell, 3, False
INFO 02:33PM [get_matrix]: Exporting matrix data to (C:\Users\slab\AppData\Loca

...

INFO 02:33PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.AnsysQ3DSetup'>)
INFO 02:33PM [analyze]: Analyzing setup Setup
INFO 02:34PM [get_matrix]: Exporting matrix data to (C:\Users\slab\AppData\Local\Temp\tmpcxv1d1xh.txt, C, , Setup:LastAdaptive, "Original", "ohm", "nH", "fF", "mSie", 5000000000, Maxwell, 1, False
INFO 02:34PM [get_matrix]: Exporting matrix data to (C:\Users\slab\AppData\Local\Temp\tmptxm77059.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5000000000, Maxwell, 1, False
INFO 02:34PM [get_matrix]: Exporting matrix data to (C:\Users\slab\AppData\Local\Temp\tmp1g3y66yl.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5000000000, Maxwell, 2, False
INFO 02:34PM [get_matrix]: Exporting matrix data to (C:\Users\slab\AppData\Local\Temp\tmpn7i69dys.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5000000000, Maxwell, 3, False
INFO 02:34PM [get_matrix]: Exporting matrix data to (C:\Users\slab\AppData\Loca

820
                    ground_main_plane  pad_left_Q1  pad_right_Q1  \
ground_main_plane           344.08657   -114.36365    -113.70412   
pad_left_Q1                -114.36365    196.98520     -63.05214   
pad_right_Q1               -113.70412    -63.05214     182.10339   
resonator_pad_0_Q1          -16.29181    -15.16845      -0.52791   

                    resonator_pad_0_Q1  
ground_main_plane            -16.29181  
pad_left_Q1                  -15.16845  
pad_right_Q1                  -0.52791  
resonator_pad_0_Q1            32.26066  


 C:\Users\slab\AppData\Local\Temp\ipykernel_16980\641277870.py: 86


[1, 2] [3]
Predicted Values

Transmon Properties
f_Q 3.936550 [GHz]
EC 154.033812 [MHz]
EJ 13.616300 [GHz]
alpha -169.889880 [MHz]
dispersion 0.017696 [KHz]
Lq 11.995161 [nH]
Cq 125.753092 [fF]
T1 642.758525 [us]

**Coupling Properties**

tCqbus1 6.827672 [fF]
gbus1_in_MHz 52.904712 [MHz]
χ_bus1 -0.216165 [MHz]
1/T1bus1 247.612341 [Hz]
T1bus1 642.758525 [us]
Bus-Bus Couplings


: 

In [ ]:
c1.sim.capacitance_matrix

,ground_main_plane,pad_left_Q1,pad_right_Q1,resonator_pad_0_Q1
ground_main_plane,267.40667,-72.36050,-74.91533,-18.22660
pad_left_Q1,-72.36050,119.83780,-31.18849,-13.19377
pad_right_Q1,-74.91533,-31.18849,110.10306,-0.55787
resonator_pad_0_Q1,-18.22660,-13.19377,-0.55787,32.30918


In [ ]:
df

In [ ]:
c1.sim.close()

In [ ]:
eig_all, renderer_hfss, hfss = dp.init_hfss_sim(nmode = 1,Ljs = [f'{qubit.L.to(u.nH).value:.2f}nH'])

In [ ]:
default_options

{'cut_l': '400um+500um+15um+25um',
 'cut_h': '1200um',
 'gap': '25um',
 'w': '90um',
 'l': '600um',
 'r': '60um',
 'cpw_l': '50um',
 'coupling_gap': '15um',
 'JJ_cutout_w': '70um',
 'JJ_cutout_h': '70um',
 'JJ_cutout_r': '150um',
 'JJ_c_contact_l': '40um',
 'JJ_c_contact_r': '2.5um',
 'JJ_c_contact_w': '10um',
 'coupling_d': '250um',
 'coupling_pad_w': '90um',
 'coupling_r': '40um',
 'cpw_pin': '10um',
 'chip': 'main',
 'resolution': '10',
 'junction': 'False',
 'orientation': '0',
 'JJ_c_contact_shortl': '10um',
 'istunnel': 'False',
 'coupling_gd': '650um'}

## Qubit options

In [103]:
w = '400um'
qubit_q1 = TransmonQubit(4*u.GHz, ratio = 65)
default_options_q1 = {'cut_l': '600um+'+w+'+'+w,
 'cut_h': '1200um',
 'gap': '70um',
 'w': w,
 'l': '650um',
 'r': w+'/2',
 'cpw_l': '100um',
 'jj_gap': '3um',
 'jj_contact_size': '3um',
 'coupling_gap': '20um',
 'JJ_cutout_w': '70um',
 'JJ_cutout_h': '70um',
 'JJ_cutout_r': '150um',
 'JJ_c_contact_l': '40um',
 'JJ_c_contact_r': '2.5um',
 'JJ_c_contact_w': '10um',
 'coupling_d': '80um',
 'coupling_pad_w': '100um',
'coupling_stub_w': '90um',
 'coupling_r': '40um',
 'cpw_pin': design.variables['trace_width'],
 'chip': 'main',
 'resolution': '10',
 'junction': 'False',
 'orientation': '0',
 'JJ_c_contact_shortl': '10um',
 'istunnel': 'False',
 'layer' : '5',
 'coupling_arm':'False', 
'jj_taper_r': '30um'}
qubit_q2 = TransmonQubit(3.2*u.GHz, ratio = 65)
default_options_q2 = {'cut_l': '600um+'+w+"+"+w,
 'cut_h': '1300um',
 'gap': '70um',
 'jj_gap': '3um',
 'jj_contact_size': '3um',
 'w': w,
 'l': '1080um',
 'r': w+'/2',
 'cpw_l': '150um',
 'coupling_gap': '20um',
 'JJ_cutout_w': '70um',
 'JJ_cutout_h': '70um',
 'JJ_cutout_r': '150um',
 'JJ_c_contact_l': '40um',
 'JJ_c_contact_r': '2.5um',
 'JJ_c_contact_w': '10um',
 'coupling_d': '70um',
 'coupling_pad_w': '200um',
'coupling_stub_w': '90um',
 'coupling_r': '40um',
 'cpw_pin': design.variables['trace_width'],
 'chip': 'main',
 'resolution': '10',
 'junction': 'False',
 'orientation': '0',
 'JJ_c_contact_shortl': '10um',
 'istunnel': 'False',
 'coupling_gd': '650um',
 'layer' : '5',
'coupling_arm':'False', 
'jj_taper_r': '30um'}

w3 = '250um'
qubit_q3 = TransmonQubit(4.8*u.GHz, ratio = 65)
default_options_q3 = {'cut_l': '450um+'+w3+'+'+w3,
 'cut_h': '850um',
 'gap': '70um',
 'jj_gap': '3um',
 'jj_contact_size': '3um',
 'w': w3,
 'l': '510um',
 'r': w3+'/2',
 'cpw_l': '100um',
 'coupling_gap': '20um',
 'JJ_cutout_w': '70um',
 'JJ_cutout_h': '70um',
 'JJ_cutout_r': '150um',
 'JJ_c_contact_l': '40um',
 'JJ_c_contact_r': '2.5um',
 'JJ_c_contact_w': '10um',
 'coupling_d': '70um',
 'coupling_pad_w': '100um',
'coupling_stub_w': '90um',
 'coupling_r': '40um',
 'cpw_pin': design.variables['trace_width'],
 'chip': 'main',
 'resolution': '10',
 'junction': 'False',
 'orientation': '0',
 'JJ_c_contact_shortl': '10um',
 'istunnel': 'False',
 'layer' : '5', 
 'coupling_arm':'False', 
'jj_taper_r': '30um'}

Optimal inductance L: 13.676453476080171 nH
Optimal inductance L: 17.095566845100272 nH
Optimal inductance L: 11.397044563400145 nH


### HFSS

In [ ]:
gui.rebuild()

11:41AM 51s WARNING [check_lengths]: For path table, component=trans3, key=trace has short segments that could cause issues with fillet. Values in (1-2)  are index(es) in shapely geometry.
11:41AM 51s WARNING [check_lengths]: For path table, component=trans3, key=cut has short segments that could cause issues with fillet. Values in (1-2)  are index(es) in shapely geometry.


In [ ]:
design.delete_all_components()

In [ ]:
default_options['junction'] = 'False'
q1  = DiffTransmonRounded(design,'Q3',options = default_options_q3)
gui.rebuild()

In [ ]:
# eig_all.sim.close()

In [ ]:
c1.sim.run(components = ['Q3'],open_terminations = [('Q3','cpw_stub')])
print(c1.sim.capacitance_matrix)
cap_mat = c1.sim.capacitance_matrix

INFO 09:38PM [connect_design]: 	Opened active design
	Design:    Design_q3d [Solution type: Q3D]
INFO 09:38PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.AnsysQ3DSetup'>)
INFO 09:38PM [analyze]: Analyzing setup Setup
INFO 09:39PM [get_matrix]: Exporting matrix data to (C:\Users\slab\AppData\Local\Temp\tmpw91qdgmq.txt, C, , Setup:LastAdaptive, "Original", "ohm", "nH", "fF", "mSie", 5000000000, Maxwell, 1, False
INFO 09:39PM [get_matrix]: Exporting matrix data to (C:\Users\slab\AppData\Local\Temp\tmpobgtoie6.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5000000000, Maxwell, 1, False
INFO 09:39PM [get_matrix]: Exporting matrix data to (C:\Users\slab\AppData\Local\Temp\tmpdw3bxqkp.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5000000000, Maxwell, 2, False
INFO 09:39PM [get_matrix]: Exporting matrix data to (C:\Users\slab\AppData\Local\Temp\tmpfmswo5jl.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5000000000, 

                    ground_main_plane  pad_left_Q3  pad_right_Q3  \
ground_main_plane           294.25188    -79.43895     -90.61669   
pad_left_Q3                 -79.43895    150.28974     -32.40946   
pad_right_Q3                -90.61669    -32.40946     127.20546   
resonator_pad_0_Q3          -31.08628    -35.46822      -0.89928   

                    resonator_pad_0_Q3  
ground_main_plane            -31.08628  
pad_left_Q3                  -35.46822  
pad_right_Q3                  -0.89928  
resonator_pad_0_Q3            68.08961  


In [ ]:
qubit = qubit_q3
res = ReadoutResonator(6.5*u.GHz, short_on_one_end = False)

In [ ]:
c1.setup.junctions = Dict({'Lj': qubit.L.to(u.nH).value, 'Cj': transmon.find_junction_capacitance(qubit.L).to(u.fF).value})
c1.setup.freq_readout = res.freq.to(u.GHz).value
c1.setup.freq_bus = []

c1.run_lom()
result = pd.DataFrame(c1.lumped_oscillator)
last_row = result

[1, 2] [3]
Predicted Values

Transmon Properties
f_Q 4.789565 [GHz]
EC 219.643512 [MHz]
EJ 14.336664 [GHz]
alpha -247.005772 [MHz]
dispersion 0.712660 [KHz]
Lq 11.392449 [nH]
Cq 88.189394 [fF]
T1 32.500221 [us]

**Coupling Properties**

tCqbus1 15.223946 [fF]
gbus1_in_MHz 174.971733 [MHz]
χ_bus1 -4.638606 [MHz]
1/T1bus1 4897.041872 [Hz]
T1bus1 32.500221 [us]
Bus-Bus Couplings


In [ ]:
# c1.sim.close()

In [ ]:
# eig_all.sim.setup.vars['Lj'] = f'{qubit.L.to(u.nH).value:.2f}nH'
dp.change_inductance([f'{(qubit_q2.L.to(u.nH).value):.2f}nH'],eig_all=eig_all,c1 = c1)
eig_all.sim.setup.vars['n_modes'] = 1
eig_all.sim.setup.n_modes = 1

In [ ]:
eig_all.sim.run( components = ['Q2'], open_terminations = [])

INFO 01:50PM [connect_project]: Connecting to Ansys Desktop API...
INFO 01:50PM [load_ansys_project]: 	Opened Ansys App
INFO 01:50PM [load_ansys_project]: 	Opened Ansys Desktop v2021.2.0
INFO 01:50PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/slab/Documents/Ansoft/
	Project:   Project49
INFO 01:50PM [connect_design]: No active design found (or error getting active design).
INFO 01:50PM [connect]: 	 Connected to project "Project49". No design detected
INFO 01:50PM [connect_design]: 	Opened active design
	Design:    Design_hfss [Solution type: Eigenmode]
WARNING 01:50PM [connect_setup]: 	No design setup detected.
WARNING 01:50PM [connect_setup]: 	Creating eigenmode default setup.
INFO 01:50PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)
INFO 01:50PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)
INFO 01:50PM [analyze]: Analyzing setup Setup
01:51PM 33s INFO [get_f_convergence]: Saved convergences to c:\Users\slab\D

In [ ]:
q = 'q2'

In [ ]:
date = get_current_time_string()
hfss_result = (eig_all.sim.convergence_f).dropna()
hfss_result.to_csv('data/'+date+f'hfss_{q}.csv')

In [ ]:
eig_all.sim.convergence_f

,re(Mode(1)) [g]
Pass [],
1,2.696116
2,3.336773
3,3.566058
4,3.660661
5,3.719212
6,3.752386
7,3.781119
8,3.803967
9,3.819114


## Tee

In [ ]:
design.delete_all_components()

In [ ]:
res.pin

<Quantity 15. um>

In [ ]:
TQ_options = dict(coupling_length='200 um',
                prime_width = design.variables['trace_width'],
               prime_gap = design.variables['trace_gap'],
               second_width = design.variables['trace_width'],
               second_gap = design.variables['trace_gap'],
               down_length = '150um',
               coupling_space = '2um',
               fillet = '30um',
               open_termination=True,
               hfss_wire_bonds = False,
               q3d_wire_bonds = False,
               layer = 5)
tee1 = CoupledLineTee(design,'tee1',options = TQ_options)
gui.rebuild()

In [ ]:
# c1.sim.run(components = ['tee1'],open_terminations = [('tee1','prime_start'),('tee1','prime_end'),('tee1','second_end')])


INFO 09:44PM [connect_project]: Connecting to Ansys Desktop API...
INFO 09:44PM [load_ansys_project]: 	Opened Ansys App
INFO 09:44PM [load_ansys_project]: 	Opened Ansys Desktop v2021.2.0
INFO 09:44PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/slab/Documents/Ansoft/
	Project:   Project48
INFO 09:44PM [connect_design]: 	Opened active design
	Design:    Design_hfss [Solution type: Eigenmode]
INFO 09:44PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)
INFO 09:44PM [connect]: 	Connected to project "Project48" and design "Design_hfss" 😀 

INFO 09:44PM [connect_design]: 	Opened active design
	Design:    Design_q3d [Solution type: Q3D]
INFO 09:44PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.AnsysQ3DSetup'>)
INFO 09:44PM [analyze]: Analyzing setup Setup
INFO 09:45PM [get_matrix]: Exporting matrix data to (C:\Users\slab\AppData\Local\Temp\tmpvipc8_cy.txt, C, , Setup:LastAdaptive, "Original", "ohm", "nH", "fF", "mSie", 5000000000, Maxwe

In [ ]:
# c1.sim.capacitance_matrix

In [ ]:
# qubit1 = TransmonQubit(4*u.GHz, ratio = 65)
# res1 = ReadoutResonator(6*u.GHz, short_on_one_end = False)
# dr1 = DispersiveReadout(qubit1,res1,5*u.ms)

Optimal inductance L: 13.676453476080171 nH


In [32]:
design.delete_all_components()

# Write design

In [33]:
from qiskit_metal.qlibrary.tlines.mixed_path import RouteMixed
from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight

from qiskit_metal.qlibrary.tlines.anchored_path import RouteAnchors
from qiskit_metal.qlibrary.terminations.launchpad_wb import LaunchpadWirebond
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee
from LaunchpadWirebondCustom import LaunchpadWirebondCustom


from qiskit_metal.qlibrary.lumped.cap_n_interdigital import CapNInterdigital


from qiskit_metal.qlibrary.terminations.launchpad_wb_coupled import LaunchpadWirebondCoupled

## Wirebond Pads

In [34]:
Wirebond_opt = Dict(trace_width=design.variables['trace_width'],
                           trace_gap=design.variables['trace_gap'],
                           lead_length='40.5um',
                           pad_width='500um',
                           pad_height='300um',
                           pad_gap='100um',
                           taper_height='100um',
                           layer = str(qubit_layer),
                           pin_space = '200um')

Wirebond_opt['pos_x'] = '-2450um'
Wirebond_opt['pos_y'] = '2100um'
Wirebond_opt['orientation'] = '270'

wb_left = LaunchpadWirebondCustom(design, 'wb_left', options = Wirebond_opt)
Wirebond_opt['pos_x'] = '2450um'
wb_right = LaunchpadWirebondCustom(design, 'wb_right', options = Wirebond_opt)
gui.rebuild()

In [35]:

# launch_options_l = dict(pos_x='-2650um', 
#                       pos_y='2812um', 
#                       orientation='270', 
#                       lead_length='70um', 
#                       trace_width = design.variables['trace_width'], 
#                       trace_gap = design.variables['cpw_gap'], 
#                       pos_x_end = '-2500um', 
#                       pos_y_end = '2500um',
#                       layer = 5)
# # Wirebond_opt['x_pos'] = '-2650um'
# # Wirebond_opt['y_pos'] = '2812um'
# # Wirebond_opt['orientation'] = '270'
# lp_left = LaunchpadWirebondCoupled(design, 'LP1', options = launch_options_l)
# gui.rebuild()

In [36]:
# launch_options_r = dict(pos_x='2650um', 
#                       pos_y='2812um', 
#                       orientation='270', 
#                       lead_length='70um', 
#                       trace_width = design.variables['trace_width'], 
#                       trace_gap = design.variables['cpw_gap'], 
#                       pos_x_end = '2500um', 
#                       pos_y_end = '2500um',
#                       layer = 5)
# lp_right = LaunchpadWirebondCoupled(design, 'LP2', options = launch_options_r)
# gui.rebuild()

## Tee

In [37]:
TQ_options = dict(coupling_length='400 um + 90um',
                prime_width = design.variables['trace_width'],
               prime_gap = design.variables['trace_gap'],
               second_width = design.variables['trace_width'],
               second_gap = design.variables['trace_gap'],
               down_length = '150um',
               coupling_space = '5um',
               fillet = '90um',
               open_termination=True,
               hfss_wire_bonds = False,
               q3d_wire_bonds = False,
               pos_x = '-1800um',
               pos_y = '1800um', 
               mirror = True,
               layer = 5)
tee1 = CoupledLineTee(design,'tee1',options = TQ_options)
gui.rebuild()

In [38]:
TQ_options = dict(coupling_length='400 um + 90um',
                prime_width = design.variables['trace_width'],
               prime_gap = design.variables['trace_gap'],
               second_width = design.variables['trace_width'],
               second_gap = design.variables['trace_gap'],
               down_length = '150um',
               coupling_space = '5um',
               fillet = '90um',
               open_termination=True,
               hfss_wire_bonds = False,
               q3d_wire_bonds = False,
               pos_x = '1850um',
               pos_y = '1800um', 
               mirror = False,
               layer = 5)
tee2 = CoupledLineTee(design,'tee2',options = TQ_options)
gui.rebuild()

In [39]:
TQ_options = dict(coupling_length='400 um + 90 um',
                prime_width = design.variables['trace_width'],
               prime_gap = design.variables['trace_gap'],
               second_width = design.variables['trace_width'],
               second_gap = design.variables['trace_gap'],
               down_length = '150um',
               coupling_space = '5um',
               fillet = '90um',
               open_termination=True,
               hfss_wire_bonds = False,
               q3d_wire_bonds = False,
               pos_x = '1200um',
               pos_y = '1800um', 
               mirror = False,
               layer= 5)
tee3 = CoupledLineTee(design,'tee3',options = TQ_options)
gui.rebuild()

## Qubit

In [43]:
default_options_q3['pos_x'] = '-500um'
default_options_q3['pos_y'] = '1000um'
default_options_q3['orientation'] = '0'
q1 = DiffTransmonRounded(design,'Q3',options = default_options_q3)
gui.rebuild()

In [44]:
default_options_q2['pos_x'] = '-1000um'
default_options_q2['pos_y'] = '-2000um'
default_options_q2['orientation'] = '180'
q2 = DiffTransmonRounded(design,'Q2',options = default_options_q2)
gui.rebuild()

In [45]:
default_options_q1['pos_x'] = '-900um'
default_options_q1['pos_y'] = '-700um'
default_options_q1['orientation'] = '180'
q3 = DiffTransmonRounded(design,'Q1',options = default_options_q1)
gui.rebuild()

## CPW

In [46]:
res_1 = ReadoutResonator(6.5*u.GHz, short_on_one_end = False)

In [47]:
route_options = dict(layer = 5,hfss_wire_bonds = False, 
                     fillet = '90um',
                     total_length = f'{res_1.len}'+'-1000um',
                     pin_inputs = dict(start_pin = dict(component = 'Q3', pin = 'cpw_stub'), 
                                                                end_pin = dict(component = 'tee1', pin = 'second_end')))
route_options['pin_width'] = design.variables['trace_width']
route_options['gap_width'] = design.variables['trace_gap']
route_options['trace_width'] = design.variables['trace_width']
route_options['trace_gap'] = design.variables['trace_gap']
route_options['lead'] = dict(start_straight = '300 um', end_straight = '200 um')
res1 = RouteMeander(design,name = 'res1',options = route_options)
gui.rebuild()

In [48]:
res_2 = ReadoutResonator(5.5*u.GHz, short_on_one_end = False)

In [49]:
route_options = dict(layer = 5,hfss_wire_bonds = False, 
                     fillet = '80um',
                     total_length = f'{res_2.len}'+ '-700um',
                     pin_inputs = dict(start_pin = dict(component = 'Q2', pin = 'cpw_stub'), 
                                                                end_pin = dict(component = 'tee2', pin = 'second_end')))
route_options['pin_width'] = design.variables['trace_width']
route_options['gap_width'] = design.variables['trace_gap']
route_options['trace_width'] = design.variables['trace_width']
route_options['trace_gap'] = design.variables['trace_gap']
route_options['lead'] = dict(start_straight = '2900 um', end_straight = '900 um')
res2 = RouteMeander(design,name = 'res2',options = route_options)
gui.rebuild()

In [50]:
res_3 = ReadoutResonator(6*u.GHz, short_on_one_end = False)

In [51]:
route_options = dict(layer = 5,hfss_wire_bonds = False, 
                     fillet = '90um',
                     total_length = f'{res_3.len}'+ '-700um',
                     pin_inputs = dict(start_pin = dict(component = 'Q1', pin = 'cpw_stub'), 
                                                                end_pin = dict(component = 'tee3', pin = 'second_end')))
route_options['pin_width'] = design.variables['trace_width']
route_options['gap_width'] = design.variables['trace_gap']
route_options['trace_width'] = design.variables['trace_width']
route_options['trace_gap'] = design.variables['trace_gap']
route_options['lead'] = dict(start_straight = '2100 um', end_straight = '400 um')
res3 = RouteMeander(design,name = 'res3',options = route_options)
gui.rebuild()

In [52]:
trans_ops = dp.trans_options

In [53]:
trans_ops['pin_inputs'] = dict(start_pin = dict(component = 'wb_left', pin = 'tie'),end_pin = dict(component = 'tee1', pin = 'prime_start'))
trans_ops['fillet'] = '100um'
trans1 = RouteMixed(design,name = 'trans1',options = trans_ops)
gui.rebuild()

In [54]:
trans_ops['pin_inputs'] = dict(start_pin = dict(component = 'tee3', pin = 'prime_start'),end_pin = dict(component = 'tee1', pin = 'prime_end'))
trans2 = RouteStraight(design,name = 'trans2',options = trans_ops)
gui.rebuild()

In [55]:
trans_ops['pin_inputs'] = dict(start_pin = dict(component = 'tee3', pin = 'prime_end'),end_pin = dict(component = 'tee2', pin = 'prime_start'))
trans3 = RouteStraight(design,name = 'trans3',options = trans_ops)
gui.rebuild()

11:40AM 26s WARNING [check_lengths]: For path table, component=trans3, key=trace has short segments that could cause issues with fillet. Values in (1-2)  are index(es) in shapely geometry.
11:40AM 26s WARNING [check_lengths]: For path table, component=trans3, key=cut has short segments that could cause issues with fillet. Values in (1-2)  are index(es) in shapely geometry.
11:40AM 26s WARNING [check_lengths]: For path table, component=trans3, key=trace has short segments that could cause issues with fillet. Values in (1-2)  are index(es) in shapely geometry.
11:40AM 26s WARNING [check_lengths]: For path table, component=trans3, key=cut has short segments that could cause issues with fillet. Values in (1-2)  are index(es) in shapely geometry.


In [56]:
trans_ops['pin_inputs'] = dict(start_pin = dict(component = 'wb_right', pin = 'tie'),end_pin = dict(component = 'tee2', pin = 'prime_end'))
trans_ops['fillet'] = '100um'
trans4 = RouteMixed(design,name = 'trans4',options = trans_ops)
gui.rebuild()

11:40AM 27s WARNING [check_lengths]: For path table, component=trans3, key=trace has short segments that could cause issues with fillet. Values in (1-2)  are index(es) in shapely geometry.
11:40AM 27s WARNING [check_lengths]: For path table, component=trans3, key=cut has short segments that could cause issues with fillet. Values in (1-2)  are index(es) in shapely geometry.


# EPR simulations

In [ ]:
import datetime as dt

In [ ]:
def date():
    return dt.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')

In [ ]:
qubit = qubit_q1
eig_all, renderer_hfss, hfss = dp.init_hfss_sim(nmode = 2,Ljs = [f'{qubit.L.to(u.nH).value:.2f}nH'], conv_threshold=0.5)


### Q1

In [ ]:
qubit = qubit_q1
Lj = str(qubit.L.to(u.nH).value)+'nH'
Cj = str(transmon.find_junction_capacitance(qubit.L).to(u.fF).value)+'fF'

eig_all.sim.renderer.options['Lj'] = Lj
eig_all.sim.renderer.options['Cj'] = Cj

eig_all.sim.setup.vars = {}
eig_all.sim.setup.vars['Lj'] = Lj
eig_all.sim.setup.vars['Cj'] = Cj
components = ['Q1','res3', 'tee3']
eig_all.sim.run(name = 'q1', components = components)
dat = {}
convergence = pd.read_csv('hfss_eig_f_convergence.csv')
conv = convergence.dropna()

ind = list(conv.keys())[1:]
for i in range(2):
    freq = conv[ind[(i)]].values[-1]
    dat['Freq'+str(i+1)+'(GHz)'] = freq
    
target_freq = 4
dat['f_target'] = target_freq
# default_options['frequency'] = target_freq
dat['Lj'] = Lj
dat['Cj'] = Cj

try:
    del eig_all.setup.junctions['jj']
except:
    print('saddd')
eig_all.setup.junctions.jj1 = Dict(rect='JJ_rect_Lj_Q1_rect_jj', line='JJ_Lj_Q1_rect_jj_',
                Lj_variable='Lj', Cj_variable='Cj')
eig_all.run_epr()
chi = eig_all.sim.renderer.epr_quantum_analysis.get_chis().to_numpy()
freq_EPR= eig_all.sim.renderer.epr_quantum_analysis.get_frequencies().to_numpy()[:,0]

# renderer_hfss.clean_active_design()
for i in range(2):
    freq = freq_EPR[i]
    dat['Freq_EPR'+str(i+1)+'(MHz)'] = freq
    for j in range(i+1):
        dat['Chi'+str(i+1)+'_'+str(j+1)+'(MHz)'] = chi[i][j] 



data = pd.DataFrame(dat, index = [0])

data.to_csv(date()+'q1.csv')

INFO 10:58AM [connect_design]: 	Opened active design
	Design:    q1_hfss [Solution type: Eigenmode]
INFO 10:58AM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)
INFO 10:58AM [analyze]: Analyzing setup Setup
11:00AM 03s INFO [get_f_convergence]: Saved convergences to c:\Users\slab\Desktop\Wendy-qiskit-code\Wendy-masklib-code\hfss_eig_f_convergence.csv


saddd
Design "q1_hfss" info:
	# eigenmodes    2
	# variations    1
Design "q1_hfss" info:
	# eigenmodes    2
	# variations    1


 c:\Users\slab\anaconda3\envs\qmetal\lib\site-packages\pyEPR\project_info.py: 239



        energy_elec_all       = 1.47596519608343e-23
        energy_elec_substrate = 1.34183301689044e-23
        EPR of substrate = 90.9%

        energy_mag    = 1.47537746804752e-23
        energy_mag % of energy_elec_all  = 100.0%
        

Variation 0  [1/1]


 c:\Users\slab\anaconda3\envs\qmetal\lib\site-packages\pyEPR\core_distributed_analysis.py: 1101
 c:\Users\slab\anaconda3\envs\qmetal\lib\site-packages\pyEPR\core_distributed_analysis.py: 1102
 c:\Users\slab\anaconda3\envs\qmetal\lib\site-packages\pyEPR\core_distributed_analysis.py: 1245


...

###### Numerical diagonalization

Lj,13.68
0,3992.05
1,5930.27


#### Kerr Non-linear coefficient table (MHz)

###### Numerical diagonalization

0         1
Lj                       
13.68 0  196.36  4.58e-01
      1    0.46  3.04e-04

### Q3

In [ ]:
qubit = qubit_q3
components = ['Q3','res1', 'tee1']


dat = {}
convergence = pd.read_csv('hfss_eig_f_convergence.csv')
conv = convergence.dropna()

ind = list(conv.keys())[1:]
for i in range(2):
    freq = conv[ind[(i)]].values[-1]
    dat['Freq'+str(i+1)+'(GHz)'] = freq
    
target_freq = 4.8
dat['f_target'] = target_freq

Lj = str(qubit.L.to(u.nH).value)+'nH'
Cj = str(transmon.find_junction_capacitance(qubit.L).to(u.fF).value)+'fF'
eig_all.sim.renderer.options['Lj'] = Lj
eig_all.sim.renderer.options['Cj'] = Cj

eig_all.sim.setup.vars = {}
eig_all.sim.setup.vars['Lj'] = Lj
eig_all.sim.setup.vars['Cj'] = Cj
eig_all.sim.run(name = 'q3', components = components)

dat['Lj'] = Lj
dat['Cj'] = Cj

try:
    del eig_all.setup.junctions['jj']
except:
    print('saddd')
eig_all.setup.junctions.jj1 = Dict(rect='JJ_rect_Lj_Q3_rect_jj', line='JJ_Lj_Q3_rect_jj_',
                Lj_variable='Lj', Cj_variable='Cj')
eig_all.run_epr()
chi = eig_all.sim.renderer.epr_quantum_analysis.get_chis().to_numpy()
freq_EPR= eig_all.sim.renderer.epr_quantum_analysis.get_frequencies().to_numpy()[:,0]

# renderer_hfss.clean_active_design()
for i in range(2):
    freq = freq_EPR[i]
    dat['Freq_EPR'+str(i+1)+'(MHz)'] = freq
    for j in range(i+1):
        dat['Chi'+str(i+1)+'_'+str(j+1)+'(MHz)'] = chi[i][j] 



data = pd.DataFrame(dat, index = [0])

data.to_csv(date()+'q3.csv')

INFO 10:52AM [connect_design]: 	Opened active design
	Design:    q3_hfss [Solution type: Eigenmode]
INFO 10:52AM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)
INFO 10:52AM [analyze]: Analyzing setup Setup
10:53AM 12s INFO [get_f_convergence]: Saved convergences to c:\Users\slab\Desktop\Wendy-qiskit-code\Wendy-masklib-code\hfss_eig_f_convergence.csv


saddd
Design "q3_hfss" info:
	# eigenmodes    2
	# variations    1
Design "q3_hfss" info:
	# eigenmodes    2
	# variations    1


 c:\Users\slab\anaconda3\envs\qmetal\lib\site-packages\pyEPR\project_info.py: 239



        energy_elec_all       = 8.71498859550872e-24
        energy_elec_substrate = 7.90651123094568e-24
        EPR of substrate = 90.7%

        energy_mag    = 8.70248353879333e-24
        energy_mag % of energy_elec_all  = 99.9%
        

Variation 0  [1/1]


 c:\Users\slab\anaconda3\envs\qmetal\lib\site-packages\pyEPR\core_distributed_analysis.py: 1101
 c:\Users\slab\anaconda3\envs\qmetal\lib\site-packages\pyEPR\core_distributed_analysis.py: 1102
 c:\Users\slab\anaconda3\envs\qmetal\lib\site-packages\pyEPR\core_distributed_analysis.py: 1245


...

###### Numerical diagonalization

Lj,11.4
0,4595.55
1,6706.49


#### Kerr Non-linear coefficient table (MHz)

###### Numerical diagonalization

0         1
Lj                      
11.4 0  216.78  8.72e-01
     1    0.87  9.99e-04

### Q2

In [ ]:
eig_all.sim.renderer.options

{'Lj': '17.095566845100272nH',
 'Cj': '3.224534868841612fF',
 '_Rj': 0,
 'max_mesh_length_jj': '7um',
 'max_mesh_length_port': '7um',
 'project_path': None,
 'project_name': None,
 'design_name': None,
 'x_buffer_width_mm': 0.5,
 'y_buffer_width_mm': 0.5,
 'wb_threshold': '72um',
 'wb_offset': '0um',
 'wb_size': 1.6,
 'wb_height': 0.7,
 'plot_ansys_fields_options': {'name': 'NAME:Mag_E1',
  'UserSpecifyName': '0',
  'UserSpecifyFolder': '0',
  'QuantityName': 'Mag_E',
  'PlotFolder': 'E Field',
  'StreamlinePlot': 'False',
  'AdjacentSidePlot': 'False',
  'FullModelPlot': 'False',
  'IntrinsicVar': "Phase='0deg'",
  'PlotGeomInfo_0': '1',
  'PlotGeomInfo_1': 'Surface',
  'PlotGeomInfo_2': 'FacesList',
  'PlotGeomInfo_3': '1'}}

In [ ]:
qubit_q2.L

<Quantity 17.09556685 nH>

In [ ]:
qubit = qubit_q2
components = ['Q2','res2', 'tee2']


dat = {}
convergence = pd.read_csv('hfss_eig_f_convergence.csv')
conv = convergence.dropna()

ind = list(conv.keys())[1:]
for i in range(2):
    freq = conv[ind[(i)]].values[-1]
    dat['Freq'+str(i+1)+'(GHz)'] = freq
    
target_freq = 3.2
dat['f_target'] = target_freq

Lj = str(qubit.L.to(u.nH).value)+'nH'
dp.change_inductance([Lj], eig_all, c1)


Cj = str(transmon.find_junction_capacitance(qubit.L).to(u.fF).value)+'fF'
eig_all.sim.renderer.options['Lj'] = Lj
eig_all.sim.renderer.options['Cj'] = Cj

eig_all.sim.setup.vars = {}
eig_all.sim.setup.vars['Lj'] = Lj
eig_all.sim.setup.vars['Cj'] = Cj

dat['Lj'] = Lj
dat['Cj'] = Cj
eig_all.sim.run(name = 'q2', components = components)
try:
    del eig_all.setup.junctions['jj']
except:
    print('saddd')
eig_all.setup.junctions.jj1 = Dict(rect='JJ_rect_Lj_Q2_rect_jj', line='JJ_Lj_Q2_rect_jj_',
                Lj_variable='Lj', Cj_variable='Cj')
eig_all.run_epr()
chi = eig_all.sim.renderer.epr_quantum_analysis.get_chis().to_numpy()
freq_EPR= eig_all.sim.renderer.epr_quantum_analysis.get_frequencies().to_numpy()[:,0]

# renderer_hfss.clean_active_design()
for i in range(2):
    freq = freq_EPR[i]
    dat['Freq_EPR'+str(i+1)+'(MHz)'] = freq
    for j in range(i+1):
        dat['Chi'+str(i+1)+'_'+str(j+1)+'(MHz)'] = chi[i][j] 



data = pd.DataFrame(dat, index = [0])

data.to_csv(date()+'q2.csv')

INFO 10:50AM [connect_design]: 	Opened active design
	Design:    q2_hfss [Solution type: Eigenmode]
INFO 10:50AM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)
INFO 10:50AM [analyze]: Analyzing setup Setup
10:51AM 28s INFO [get_f_convergence]: Saved convergences to c:\Users\slab\Desktop\Wendy-qiskit-code\Wendy-masklib-code\hfss_eig_f_convergence.csv


saddd
Design "q2_hfss" info:
	# eigenmodes    2
	# variations    1
Design "q2_hfss" info:
	# eigenmodes    2
	# variations    1


 c:\Users\slab\anaconda3\envs\qmetal\lib\site-packages\pyEPR\project_info.py: 239



        energy_elec_all       = 8.75623307126701e-24
        energy_elec_substrate = 7.95192677412188e-24
        EPR of substrate = 90.8%

        energy_mag    = 8.74693749870211e-24
        energy_mag % of energy_elec_all  = 99.9%
        

Variation 0  [1/1]


 c:\Users\slab\anaconda3\envs\qmetal\lib\site-packages\pyEPR\core_distributed_analysis.py: 1101
 c:\Users\slab\anaconda3\envs\qmetal\lib\site-packages\pyEPR\core_distributed_analysis.py: 1102
 c:\Users\slab\anaconda3\envs\qmetal\lib\site-packages\pyEPR\core_distributed_analysis.py: 1245


...

###### Numerical diagonalization

Lj,17.1
0,3172.99
1,5355.58


#### Kerr Non-linear coefficient table (MHz)

###### Numerical diagonalization

0         1
Lj                      
17.1 0  154.22  3.35e-01
     1    0.33  1.97e-04

## Junction probe pads

In [57]:
from bandaged_dolan import DolanJunctionBandage
from probe_pad import ProbePad
from text import Text_object

In [405]:
maximum_jj_w = 500 #nm
small_jj_length = 20#nm

h_pocket = 300 #um
w_pocket = 800





dose_array_options = Dict(
    pos_x = '0um', 
    pos_y = '0um',
    w_pocket = f'{w_pocket:.0f} um',
    h_pocket = f'{h_pocket:.0f} um',
    w_pad = '300 um',
    h_pad = '200 um',
    pad_gap = '7 um',
    orientation = '0',
    layer = f'{qubit_layer}',
    )

text_options = Dict(
        pos_x = '0um', 
        pos_y = '0um',
        text = 'haha',
        text_height = '20 mm',
        orientation = '0',
        layer = f'{qubit_layer}',
        subtract = 'False'
        )



JJ_options = Dict(
    pos_x = '0um',
    pos_y = '0um',
    w_pad_pin = '2 um',
    maximum_jj_width = '950nm',
    small_jj_length = '400nm',
    w_pad_u = '3 um',
    d_pin = '250 nm',
    d_u = '550 nm',
    total_length = '1 um',
    w_top_pin = '550 nm',
    w_top_u = '550 nm',
    w_bot_pin = '550 nm',
    w_bot_u = '550 nm',
    top_bot_offset = '0 nm' ,
    jj_extra = '600nm',
    jj_gap = '0.2um',
    fillet = '30 nm',
    # jj_gap_actual = '0.2um',
    # Lj = '10',
    resolution = '5',
    bandage = 'True',
    bandage_h = '20 um',
    bandage_w = '20 um',
    bandage_layer = '299',
    bandage_uc_layer = '699',
    bandage_loc = '0.5um',
    # Jc = '0.1',
    orientation = '270',
    pin_layer = f'{junction_layer}',
    gap_layer = f'{junction_area_layer}',
    area_layer_opt = 'False',
    jj_orientation = '0',
    tapered = 'False',
    taper_r = '10um',
    uc_override_band = '1um',)

JJ_options1 = JJ_options.copy()
JJ_options1['d_pin'] = '10um'
JJ_options1['d_u'] = '12um'
JJ_options1['w_pad_pin'] = '5um'
JJ_options1['w_pad_u'] = '6um'
JJ_options1['total_length'] = '30um'
JJ_options1['jj_extra'] = '0um'
JJ_options1['small_jj_length'] = '600nm'


def junction_probe_pad(x,y,jj_w,jj_gap, name, dose_array_options = dose_array_options, JJ_options = JJ_options):

    dose_array_options['pos_x'] = x
    dose_array_options['pos_y'] = y

    JJ_options['pos_x'] = x
    JJ_options['pos_y'] = y
    JJ_options['w_bot_pin'] = jj_w
    JJ_options['jj_gap'] = jj_gap

    dose_array = ProbePad(design, name+'ProbePad', dose_array_options)
    dolan = DolanJunctionBandage(design,name+'JJ',JJ_options)
    text_options['pos_x'] = x+f'-{w_pocket/2}um'
    text_options['pos_y'] = y #+ f'-{w_pocket/2}um'
    text_options['text'] = name
    text_options['text_height'] = '50um'
    text_options['orientation'] = '0'
    Text_object(design, name+'text',text_options)




## Junction

In [35]:
import Transmon_specifications as jj

### Parameters

In [36]:
offset = 0
Jc = 0.15*u.uA/u.um**2*(1+offset)
overlap = 220*u.nm

jj1_len = (jj.find_junction_area(qubit_q1.L, Jc = Jc)/overlap).to(u.nm)


jj2_len = (jj.find_junction_area(qubit_q2.L, Jc = Jc)/overlap).to(u.nm)


jj3_len = (jj.find_junction_area(qubit_q3.L, Jc = Jc)/overlap).to(u.nm)


In [ ]:
Jc = 0.55*u.uA/u.um**2
jj_lens = [0.55*u.um, 0.55*u.um, 0.55*u.um]
qubits = [qubit_q2,qubit_q1,qubit_q3]
for i in range(1):
    Lj, Ej = jj.find_junction_inductance(jj_lens[i],overlap,Jc)
    print('target: ',qubits[i].freq, '; actual:',np.sqrt(8*Ej*qubits[i].EC).to(u.GHz))
    print('Ej', Ej.to(u.GHz))
   

target:  3.200000000000175 GHz ; actual: 4.453377660306013 GHz
Ej 33.054287642187774 GHz


In [86]:
Lj, Ej = jj.find_junction_inductance(jj_lens[0],overlap,Jc)
Ec = 70*u.MHz
# Create a Transmon object
transmon = scq.Transmon(Ej.to(u.GHz).value, Ec.to(u.GHz).value, ng=0, ncut=30, truncated_dim=10)

# Calculate the eigenenergies
eigenenergies = transmon.eigenvals()

# Calculate the transition frequencies (in GHz)
transition_frequencies = np.diff(eigenenergies)

# Calculate the anharmonicity (in GHz)
anharmonicity = transition_frequencies[1] - transition_frequencies[0]


In [87]:
transition_frequencies

array([4.23118093, 4.15841157, 4.08360539, 4.00656102, 3.92704197])

In [88]:
anharmonicity

-0.0727693624787733

In [599]:
np.sqrt(8*Ej*qubit_q3.EC).si

<Quantity 6.58783054e+09 1 / s>

In [553]:
d_pin = 15
u_pin = 0.75
JJ_options1 = JJ_options.copy()
JJ_options1['d_pin'] = f'{d_pin}um'
JJ_options1['d_u'] = f'{d_pin+2*u_pin*2}um'
JJ_options1['w_pad_pin'] = '2.5um'
JJ_options1['w_pad_u'] = '3um'
JJ_options1['total_length'] = f'{q1.options.jj_gap}+{d_pin}um+{u_pin*2}um'
JJ_options1['jj_extra'] = '0um'
JJ_options1['small_jj_length'] = '600nm'

JJ_options1['bandage_h'] = '30um'
JJ_options1['bandage_w'] = '5um'

JJ_options1['bandage_loc'] = '7um'

In [554]:
def junction_on_qubit(x,y,jj_w,jj_gap,gap, name,  JJ_options = JJ_options):
    JJ_options['pos_x'] = x
    JJ_options['pos_y'] = y
    JJ_options['w_bot_pin'] = jj_w
    JJ_options['jj_gap'] = jj_gap
    JJ_options['total_length'] = gap
    dolan = DolanJunctionBandage(design,name+'JJ',JJ_options)


In [555]:
jjw = '200nm'

### Junction on qubit

In [556]:
from rounded_rectangle import rounded_rec as rec2

In [557]:
taper_r = 50
w_pad_pin = 10
d_pin = 20
uc = 1
pad = rec2(w_pad_pin, 
            taper_r, 
            same_radius = False,  
            r1 = 0,
            r2 = 0,
            r3 = taper_r,
            r4 = taper_r,
            resolution = int(taper_r/0.002),
            d1 = [-1,-1],
            d2 = [-1,1],
            d3 = [-1,1],
            d4 = [1,1]
            )
pad = draw.translate(pad, 0, taper_r/2)
rectangle = rec2(w_pad_pin*100, d_pin*100,)
rectangle = draw.translate(rectangle, 0, d_pin*100/2)
pad_pin = pad.difference(rectangle)
pad_u = draw.shapely.buffer(pad, uc)
pad_u = pad_u.difference(pad)

In [558]:
# draw.show(pad_pin)

In [559]:
junction_on_qubit(q1.get_jj_location()[0],
                  q1.get_jj_location()[1],
                  f'{jj1_len.to(u.nm).value:.2f}nm',
                  jjw,
                  q1.options['jj_gap']+'+'+JJ_options1['d_u']+'*2',
                  'q1',
                  JJ_options = JJ_options1)
junction_on_qubit(q2.get_jj_location()[0],
                  q2.get_jj_location()[1],
                  f'{jj2_len.to(u.nm).value:.2f}nm',
                  jjw,
                  q2.options['jj_gap']+'+'+JJ_options1['d_u']+'*2',
                  'q2',
                  JJ_options = JJ_options1)

junction_on_qubit(q3.get_jj_location()[0],
                  q3.get_jj_location()[1],
                  f'{jj3_len.to(u.nm).value:.2f}nm',
                  jjw,
                  q3.options['jj_gap']+'+'+JJ_options1['d_u']+'*2',
                  'q3',
                  JJ_options = JJ_options1)
gui.rebuild()

01:07PM 23s WARNING [check_lengths]: For path table, component=trans3, key=trace has short segments that could cause issues with fillet. Values in (1-2)  are index(es) in shapely geometry.
01:07PM 23s WARNING [check_lengths]: For path table, component=trans3, key=cut has short segments that could cause issues with fillet. Values in (1-2)  are index(es) in shapely geometry.


### Junction Probe Pads

In [560]:
chip_size = 6800
max_xy = chip_size/2
JJ_options['small_jj_length'] = '600nm'

junction_probe_pad(f'-{-max_xy+w_pocket/2}um',f'{-max_xy+h_pocket/2}um',
                   f'{jj1_len.to(u.nm).value:.2f}nm',
                   jjw,
                   '1A',dose_array_options = dose_array_options, JJ_options = JJ_options1)
junction_probe_pad(f'-{-max_xy+w_pocket/2}um',f'{-max_xy+h_pocket/2+h_pocket*1.1}um',
                   f'{jj1_len.to(u.nm).value:.2f}nm',
                   jjw,
                   '1B',dose_array_options = dose_array_options, JJ_options = JJ_options1)
junction_probe_pad(f'-{-max_xy+w_pocket/2}um',f'{-max_xy+h_pocket/2+h_pocket*2.2}um',
                   f'{jj2_len.to(u.nm).value:.2f}nm',
                   jjw,
                   '2A',dose_array_options = dose_array_options, JJ_options = JJ_options1)

junction_probe_pad(f'-{-max_xy+w_pocket/2}um',f'{-max_xy+h_pocket/2+ h_pocket*3.3}um',
                   f'{jj2_len.to(u.nm).value:.2f}nm',
                   jjw,
                   '2B',dose_array_options = dose_array_options, JJ_options = JJ_options1)
junction_probe_pad(f'-{-max_xy+w_pocket/2}um',f'{-max_xy+h_pocket/2+h_pocket*4.4}um',
                   f'{jj3_len.to(u.nm).value:.2f}nm',
                   jjw,
                   '3A',dose_array_options = dose_array_options, JJ_options = JJ_options1)
junction_probe_pad(f'-{-max_xy+w_pocket/2}um',f'{-max_xy+h_pocket/2+h_pocket*5.5}um',
                   f'{jj3_len.to(u.nm).value:.2f}nm',
                   jjw,
                   '3B',dose_array_options = dose_array_options, JJ_options = JJ_options1)

In [561]:
gui.rebuild()

01:07PM 25s WARNING [check_lengths]: For path table, component=trans3, key=trace has short segments that could cause issues with fillet. Values in (1-2)  are index(es) in shapely geometry.
01:07PM 25s WARNING [check_lengths]: For path table, component=trans3, key=cut has short segments that could cause issues with fillet. Values in (1-2)  are index(es) in shapely geometry.


# GDS

In [562]:
# design.chips.main.size['size_x'] = '10mm'
# design.chips.main.size['size_y'] = '10mm'
# gui.rebuild()
a_gds = design.renderers.gds

In [563]:
qubit_layer = 5
junction_layer = 20
ab_layer = 31
ab_square_layer = 30
junction_area_layer = 60
junction_bandage_layer = 299    
junction_uc_layer = 699

In [564]:
a_gds.options['max_points'] = 3000
a_gds.options['fabricate'] = True

In [565]:
a_gds.options['cheese']['view_in_file']['main'][qubit_layer] = True
a_gds.options['no_cheese']['view_in_file']['main'][qubit_layer] = True
a_gds.options['cheese']['view_in_file']['main'][junction_layer] = False
a_gds.options['no_cheese']['view_in_file']['main'][junction_layer] = False
a_gds.options['cheese']['view_in_file']['main'][ab_layer] = False
a_gds.options['no_cheese']['view_in_file']['main'][ab_layer] = False
a_gds.options['cheese']['view_in_file']['main'][ab_square_layer] = False
a_gds.options['no_cheese']['view_in_file']['main'][ab_square_layer] = True
a_gds.options['cheese']['view_in_file']['main'][1] = False
a_gds.options['no_cheese']['view_in_file']['main'][1] = False

In [566]:
# a_gds.options['cheese']['view_in_file']['main'][qubit_layer] = True
# a_gds.options['no_cheese']['view_in_file']['main'][qubit_layer] = True
# a_gds.options['cheese']['view_in_file']['main'][junction_layer] = True
# a_gds.options['no_cheese']['view_in_file']['main'][junction_layer] = True
# a_gds.options['cheese']['view_in_file']['main'][ab_layer] = True
# a_gds.options['no_cheese']['view_in_file']['main'][ab_layer] = True
# a_gds.options['cheese']['view_in_file']['main'][ab_square_layer] = True
# a_gds.options['no_cheese']['view_in_file']['main'][ab_square_layer] = True
# a_gds.options['cheese']['view_in_file']['main']['1'] = True
# a_gds.options['no_cheese']['view_in_file']['main']['1'] = True



# a_gds.options['fabricate'] = False
a_gds.options.tolerance = '0.0000005'
a_gds.options.precision = '0.0000000005'

In [567]:
a_gds.options['negative_mask'] = dict(main= [qubit_layer])

In [568]:
a_gds.options

{'short_segments_to_not_fillet': 'True',
 'check_short_segments_by_scaling_fillet': '1.0',
 'gds_unit': 0.001,
 'ground_plane': 'True',
 'negative_mask': {'main': [5]},
 'fabricate': True,
 'corners': 'circular bend',
 'tolerance': '0.0000005',
 'precision': '0.0000000005',
 'width_LineString': '10um',
 'path_filename': '../resources/Fake_Junctions.GDS',
 'junction_pad_overlap': '5um',
 'max_points': 3000,
 'cheese': {'datatype': '100',
  'shape': '0',
  'cheese_0_x': '10um',
  'cheese_0_y': '10um',
  'cheese_1_radius': '10um',
  'view_in_file': {'main': {1: False,
    5: True,
    20: False,
    31: False,
    30: False}},
  'delta_x': '100um',
  'delta_y': '100um',
  'edge_nocheese': '150um'},
 'no_cheese': {'datatype': '99',
  'buffer': '200um',
  'cap_style': '2',
  'join_style': '2',
  'view_in_file': {'main': {1: False,
    5: True,
    20: False,
    31: False,
    30: True}}},
 'bounding_box_scale_x': '1.2',
 'bounding_box_scale_y': '1.2'}

In [569]:
a_gds.options.cheese.edge_nocheese = '150um'
a_gds.options.no_cheese.buffer = '200um'
a_gds.options.cheese.cheese_1_radius = '10um'
a_gds.options.cheese.cheese_0_x = '10um'
a_gds.options.cheese.cheese_0_y = '10um'

In [570]:
# gui.rebuild()
a_gds.export_to_gds(f'SiFT_offset{offset}.gds')

 c:\_lib\python\qiskit-metal\qiskit_metal\renderers\renderer_gds\gds_renderer.py: 780
 c:\_lib\python\qiskit-metal\qiskit_metal\renderers\renderer_gds\gds_renderer.py: 780
01:07PM 27s WARNING [_qgeometry_to_gds]: Unexpected shapely object geometry.The variable qgeometry_element is <class 'numpy.float64'>, method can currently handle Polygon and FlexPath.
01:07PM 27s WARNING [_qgeometry_to_gds]: Unexpected shapely object geometry.The variable qgeometry_element is <class 'numpy.float64'>, method can currently handle Polygon and FlexPath.
01:07PM 27s WARNING [_qgeometry_to_gds]: Unexpected shapely object geometry.The variable qgeometry_element is <class 'numpy.float64'>, method can currently handle Polygon and FlexPath.
01:07PM 27s WARNING [_qgeometry_to_gds]: Unexpected shapely object geometry.The variable qgeometry_element is <class 'numpy.float64'>, method can currently handle Polygon and FlexPath.
01:07PM 27s WARNING [_import_junction_gds_file]: Not able to find file:"../resources/Fak

1

In [571]:
a_gds.options['check_short_segments_by_scaling_fillet'] = '1.0'